In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW, BertTokenizerFast, Trainer, TrainingArguments, DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification
from abc import ABC, abstractmethod
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import os
from datasets import Dataset

from tensorboard import notebook



/Users/venkateshmurugadas/.pyenv/versions/3.10.0/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class DataReader:
    def __init__(self, data_folder, output_file):
        self.data_folder = data_folder
        self.output_file = output_file

    def process(self):
        files = [pd.read_xml(os.path.join(self.data_folder, file)) for file in os.listdir(self.data_folder) if file.endswith(".xml")]
        data = pd.concat(files, ignore_index=True)
        data.rename(columns={'t1': 'premise', 't2': 'hypothesis'}, inplace=True)

        # Convert unique string labels to integers
        unique_labels = data['label'].unique()
        data['labels'] = data['label']
        label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
        data['label'] = data['label'].map(label_to_int)

        data.dropna(inplace=True)
        data.reset_index(drop=True, inplace=True)
        data.to_csv(self.output_file, index=False)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, custom_data, device=None):
        self.custom_data = custom_data
        # if device:
        #     self.custom_data = [item.to(device) for item in self.custom_data]

    def __len__(self):
        return len(self.custom_data)

    def __getitem__(self, idx):
        return self.custom_data[idx]

class DataProcessor(ABC):
    def __init__(self, tokenizer, config):
        self.tokenizer = tokenizer
        self.config = config

    @abstractmethod
    def tokenize_and_cut(self, sentence):
        pass

    @abstractmethod
    def preprocess(self, premise, hypothesis, label):
        pass

    def split_dataset(self, dataset, train_val_ratio=0.9):
        train_val_split_idx = int(len(dataset) * train_val_ratio)
        train_val_dataset, test_dataset = dataset[:train_val_split_idx], dataset[train_val_split_idx:]

        train_split_idx = int(len(train_val_dataset) * train_val_ratio)
        train_dataset, val_dataset = train_val_dataset[:train_split_idx], train_val_dataset[train_split_idx:]

        return train_dataset, val_dataset, test_dataset

    @abstractmethod
    def get_data_loaders(self, csv_file):
        pass

class BERTDataProcessor(DataProcessor):
    def __init__(self, tokenizer, config):
        super().__init__(tokenizer, config)
        self.data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

    def tokenize_and_cut(self, premise, hypothesis):
        tokens = self.tokenizer(premise, hypothesis,
                                max_length=self.config.max_length,
                                truncation=True)
        return tokens

    def preprocess(self, premise, hypothesis, label):
        tokens = self.tokenize_and_cut(premise, hypothesis)
        tokens["labels"] = label
        return tokens

    def get_data_loaders(self, csv_file):
        df = pd.read_csv(csv_file)
        dataset = [{"premise": row["premise"], "hypothesis": row["hypothesis"], "label": row["label"]} for _, row in df.iterrows()]
        dataset = [self.preprocess(data["premise"], data["hypothesis"], data["label"]) for data in dataset]

        # split the dataset into training, validation and test sets

        train_dataset, val_dataset, test_dataset =  self.split_dataset(dataset)

        # train_dataset = Dataset.from_dict(train_dataset)
        # val_dataset = Dataset.from_dict(val_dataset)
        # test_dataset = Dataset.from_dict(test_dataset)

        # train_loader = DataLoader(train_dataset, batch_size=self.config.batch_size, shuffle=True, collate_fn=self.data_collator)
        # val_loader = DataLoader(val_dataset, batch_size=self.config.batch_size, shuffle=False, collate_fn=self.data_collator)
        # test_loader = DataLoader(test_dataset, batch_size=self.config.batch_size, shuffle=False, collate_fn=self.data_collator)

        return train_dataset, val_dataset, test_dataset

In [13]:
class BertConfig:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    max_length = 128
    batch_size = 8
    learning_rate = 1e-6
    num_epochs = 3
    num_labels = 2
    warmup_steps = 300
    weight_decay = 0.01
    log_steps = 10

In [9]:
# Load pretrained model and tokenizer
config  = BertConfig()
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels =config.num_labels)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [14]:
train_folder = "data/COLIEE2021statute_data-English/train"
train_out_file = 'data/coliee_train/coliee_2021.csv'
data_reader = DataReader(train_folder, train_out_file)
data_reader.process()


data_processor = BERTDataProcessor(tokenizer, config)
train_loader, val_loader, test_loader = data_processor.get_data_loaders(train_out_file)

In [15]:
%tensorboard --logdir logs

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Training the model
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    warmup_steps=config.warmup_steps,
    weight_decay=config.weight_decay,
    logging_dir='./logs',
    logging_steps=config.log_steps,  # Log every 10 steps
    # save_steps=100,  # Save the model every 100 steps
    evaluation_strategy='epoch',  # Evaluate the model every 'logging_steps'
    save_strategy='epoch',  # Save the model every epoch
    load_best_model_at_end=True,
    use_mps_device=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_loader,
    eval_dataset=val_loader,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./results")

# # You can then load the model with
# loaded_model = BertForSequenceClassification.from_pretrained("./results")




/Users/venkateshmurugadas/.pyenv/versions/3.10.0/envs/thesis/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 10/2460 [00:05<18:28,  2.21it/s]

{'loss': 0.65, 'learning_rate': 1.6666666666666667e-06, 'epoch': 0.12}


  1%|          | 20/2460 [00:09<17:44,  2.29it/s]

...

 14%|█▍        | 350/2460 [02:51<15:52,  2.22it/s]

{'loss': 0.709, 'learning_rate': 4.8842592592592595e-05, 'epoch': 4.27}


 15%|█▍        | 360/2460 [02:55<15:53,  2.20it/s]

{'loss': 0.6732, 'learning_rate': 4.8611111111111115e-05, 'epoch': 4.39}


KeyboardInterrupt: 

In [8]:
# Make predictions on the test set
predictions = trainer.predict(test_loader)

# Log predictions and metrics
# print(predictions.predictions)
# print(predictions.label_ids)
print(predictions.metrics)

100%|██████████| 11/11 [00:01<00:00,  8.85it/s]

{'test_loss': 0.7022614479064941, 'test_accuracy': 0.4691358024691358, 'test_runtime': 1.9434, 'test_samples_per_second': 41.68, 'test_steps_per_second': 5.66}
